In [1]:
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 29.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
login(token=hf_token)

In [22]:
%%writefile workers.py
import torch
import torch.multiprocessing as mp
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# Import the cache object we need to manually reconstruct
from transformers.cache_utils import DynamicCache
import time
import sys
from queue import Empty

# CONFIGURATION
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
MAX_NEW_TOKS = 1000 

# PREFILL WORKER ---
def prefill_worker_loop(request_queue: mp.Queue, decode_queue: mp.Queue, tokenizer):
    # This function runs in a separate process, pinned to CUDA:0
    
    device = torch.device("cuda:0")
    mp.current_process().name = "PREFILL-WORKER-0"
    
    print(f"[{mp.current_process().name}] Initializing on {device}...")
    model = None # Define model as None in case loading fails
    try:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_quant_type="nf4",
        )
        
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID, 
            quantization_config=quantization_config,
            torch_dtype=torch.bfloat16,
            device_map=device 
        )
        model.eval()
        print(f"[{mp.current_process().name}] Llama 3 8B (4-bit) loaded successfully on {device}.")
    except Exception as e:
        print(f"[{mp.current_process().name}] Failed to load model on {device}: {e}")
        decode_queue.put(None) # Signal decode to stop
        return

    try:
        while True:
            try:
                #Get a new request
                request_data = request_queue.get(timeout=1.0)
                
                # Check for STOP signal (None)
                if request_data is None:
                    print(f"[{mp.current_process().name}] STOP signal received. Exiting loop.")
                    break
                
                request_id, prompt_text = request_data
                
                print(f"[{mp.current_process().name}] Received request {request_id}. Prefilling {len(prompt_text)} chars...")
                start_time = time.time()

                inputs = tokenizer(
                    prompt_text, 
                    return_tensors="pt"
                ).to(device)
                
                # Run the PREFILL step
                with torch.no_grad():
                    outputs = model(**inputs, use_cache=True)

                prefill_time = time.time()
                
                # Extract the KV cache and logits
                kv_cache = outputs.past_key_values
                last_logits = outputs.logits[:, -1, :]
                
                # Unpack the DynamicCache and it to CPU for the queue.
                kv_cache_cpu = tuple(
                    (layer_cache[0].to("cpu"), layer_cache[1].to("cpu"))
                    for layer_cache in kv_cache
                )
                last_logits_cpu = last_logits.to("cpu")

                before_queueing_time = time.time()
                
                # Send to the decode worker
                decode_queue.put((request_id, kv_cache_cpu, last_logits_cpu))
                
                # This is the TTFT for the disaggregated system
                ttft = prefill_time - start_time
                print(f"[{mp.current_process().name}] Request {request_id} TTFT: {ttft:.2f}s. Sent to decode queue.")
                
            except Empty:
                continue 
            except (Exception, KeyboardInterrupt) as e:
                print(f"[{mp.current_process().name}] Error in prefill loop: {e}")
                break
    finally:
        print(f"[{mp.current_process().name}] Cleaning up...")
        # Send STOP signal to the decode worker
        decode_queue.put(None)
        if model:
            del model
        torch.cuda.empty_cache()
        print(f"[{mp.current_process().name}] Model unloaded from {device}. Exiting.")



# DECODE WORKER
def decode_worker_loop(decode_queue: mp.Queue, response_dict: dict, tokenizer):
    # This function runs in a separate process, pinned to CUDA:1
    
    device = torch.device("cuda:1")
    mp.current_process().name = "DECODE-WORKER-1"

    print(f"[{mp.current_process().name}] Initializing on {device}...")
    model = None # Define model as None in case loading fails
    try:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_quant_type="nf4",
        )
        
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID, 
            quantization_config=quantization_config,
            torch_dtype=torch.bfloat16,
            device_map=device 
        )
        model.eval()
        print(f"[{mp.current_process().name}] Llama 3 8B (4-bit) loaded successfully on {device}.")
    except Exception as e:
        print(f"[{mp.current_process().name}] Failed to load model on {device}: {e}")
        return

    try:
        while True:
            try:
                # Get a prefilled request from the prefill worker
                request_data = decode_queue.get(timeout=1.0)
                
                # Check for STOP signal (None)
                if request_data is None:
                    print(f"[{mp.current_process().name}] STOP signal received. Exiting loop.")
                    break 
                
                request_id, kv_cache_cpu, last_logits_cpu = request_data
                
                print(f"[{mp.current_process().name}] Received request {request_id} for DECODING...")
                start_time = time.time()
                
                # Reconstruct a new DynamicCache object
                kv_cache_tuple_gpu1 = tuple(
                    (layer_cache[0].to(device), layer_cache[1].to(device))
                    for layer_cache in kv_cache_cpu
                )
                kv_cache = DynamicCache()
                kv_cache.key_cache = [layer_kv[0] for layer_kv in kv_cache_tuple_gpu1]
                kv_cache.value_cache = [layer_kv[1] for layer_kv in kv_cache_tuple_gpu1]
                
                next_token_logits = last_logits_cpu.to(device)
                
                generated_ids = []
                
                with torch.no_grad():
                    for _ in range(MAX_NEW_TOKS):
                        # Sample the next token
                        next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
                        
                        # Add this check back in to match model.generate()
                        if next_token.item() == tokenizer.eos_token_id:
                            print(f"[{mp.current_process().name}] Request {request_id} hit EOS token. Stopping.")
                            break
                        
                        generated_ids.append(next_token.item())
                        
                        # Run the DECODE step
                        outputs = model(
                            input_ids=next_token, 
                            past_key_values=kv_cache, 
                            use_cache=True
                        )
                        
                        # Get the new cache and logits
                        kv_cache = outputs.past_key_values
                        next_token_logits = outputs.logits[:, -1, :] 
                
                #Decode the final text
                generated_text = tokenizer.decode(
                    generated_ids, 
                    skip_special_tokens=True
                )
                
                total_time = time.time() - start_time
                num_generated = len(generated_ids)
                
                # Calculate decode throughput (tokens per second)
                # This is the equivalent of 1/TPOT
                decode_tps = num_generated / (total_time + 1e-6) 
                
                print(f"[{mp.current_process().name}] Request {request_id} DECODED {num_generated} tokens in {total_time:.2f}s (decode_tps: {decode_tps:.1f} tok/s).")
                

                response_dict[request_id] = generated_text

            except Empty:
                continue
            except (Exception, KeyboardInterrupt) as e:
                print(f"[{mp.current_process().name}] Error in decode loop: {e}")
                if 'kv_cache' in locals():
                    print(f"[{mp.current_process().name}] Type of kv_cache that caused error: {type(kv_cache)}")
                break
    finally:
        print(f"[{mp.current_process().name}] Cleaning up...")
        if model:
            del model
        torch.cuda.empty_cache()
        print(f"[{mp.current_process().name}] Model unloaded from {device}. Exiting.")



Overwriting workers.py


In [23]:
import torch
import torch.multiprocessing as mp
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer
from kaggle_secrets import UserSecretsClient
import time
import os
import sys
import uuid
from concurrent.futures import ThreadPoolExecutor
import torch

import workers

In [24]:
#CONFIGURATION
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
MAX_NEW_TOKS = 1000

#LIGHT PREFILL / LIGHT DECODE
PROMPT_LL_1 = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Name three planets in our solar system that have rings.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

PROMPT_LL_2 = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Give me two synonyms for the word 'rapid'.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# LIGHT PREFILL / HEAVY DECODE
PROMPT_LH_1 = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Write a detailed 600-word essay on how artificial intelligence could transform education by 2035.
Discuss ethical, economic, and social implications.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

PROMPT_LH_2 = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Tell a long, descriptive story about a time traveler visiting ancient Egypt.
Include dialogue, emotions, and vivid scenery.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

PROMPT_LH_3 = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Generate an in-depth tutorial explaining how attention mechanisms work in transformer models.
Use equations, analogies, and examples.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# HEAVY PREFILL / LIGHT DECODE
PROMPT_HL_1 = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
[CONTEXT]
The following text is an excerpt from a 500-word article describing the structure and history of the Roman Republic, its Senate, and transition into the Roman Empire.
Imagine this as a long prefill input.
[/CONTEXT]
Who was the first Roman Emperor?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

PROMPT_HL_2 = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
[CONTEXT]
In this extensive passage, we describe the architecture of neural networks, including activation functions, optimization methods, and backpropagation.
The text is about 400 tokens long.
[/CONTEXT]
Name one commonly used activation function.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

PROMPT_HL_3 = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
[CONTEXT]
You are given a long transcript of an academic lecture discussing the origins of the French Revolution, with references to Rousseau, Voltaire, and economic disparity in 18th-century France.
[/CONTEXT]
In one line, state the main cause of the French Revolution.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# HEAVY PREFILL / HEAVY DECODE
PROMPT_HH_1 = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
[CONTEXT]
Imagine a detailed 1,000-token report summarizing global climate change models, historical CO₂ emissions, and their socioeconomic effects.
[/CONTEXT]
Write an equally long policy proposal (800–1000 words) addressing the transition to renewable energy in developing countries.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

PROMPT_HH_2 = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
[CONTEXT]
You have just read a 700-token historical overview of the Industrial Revolution.
[/CONTEXT]
Compose a comprehensive essay connecting the Industrial Revolution to modern AI-driven automation, covering technological parallels and labor implications (700+ words).
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# Unified benchmark workload
PROMPTS_TO_RUN = [
    ("p1_light_light", PROMPT_LL_1),
    ("p2_light_light", PROMPT_LL_2),
    ("p3_light_heavy", PROMPT_LH_1),
    ("p4_light_heavy", PROMPT_LH_2),
    ("p5_light_heavy", PROMPT_LH_3),
    ("p6_heavy_light", PROMPT_HL_1),
    ("p7_heavy_light", PROMPT_HL_2),
    ("p8_heavy_light", PROMPT_HL_3),
    ("p9_heavy_heavy", PROMPT_HH_1),
    ("p10_heavy_heavy", PROMPT_HH_2),
]


In [25]:
# ORCHESTRATOR API CLASS
class OrchestratorAPI:
    def __init__(self, hf_token):
        print("[ORCHESTRATOR] Initializing...")
        os.environ["HF_TOKEN"] = hf_token
        
        # Set the multiprocessing start method to 'spawn' for CUDA
        mp.set_start_method("spawn", force=True)
        
        # Create shared queues for communication
        self.request_queue = mp.Queue()
        self.decode_queue = mp.Queue()
        
        self.manager = mp.Manager()
        self.response_dict = self.manager.dict()
        
        # Load tokenizer in the main process and pass it
        self.tokenizer = AutoTokenizer.from_pretrained(workers.MODEL_ID)
        
        self.prefill_process = None
        self.decode_process = None
        print("[ORCHESTRATOR] Initialization complete.")

    def start_workers(self):
        #Starts the two worker processes
        
        print("[ORCHESTRATOR] Starting worker processes...")
        
        # Start PREFILL worker
        self.prefill_process = mp.Process(
            target=workers.prefill_worker_loop,
            args=(self.request_queue, self.decode_queue, self.tokenizer)
        )
        self.prefill_process.start()
        
        # Start DECODE worker
        self.decode_process = mp.Process(
            target=workers.decode_worker_loop,
            args=(self.decode_queue, self.response_dict, self.tokenizer)
        )
        self.decode_process.start()
        
        print("[ORCHESTRATOR] Workers started. Waiting for model loading...")
        # Simple wait - Give 2 mins for both models to load
        time.sleep(120) 
        print("[ORCHESTRATOR] Models should be loaded. System is READY.")

    def stop_workers(self):
        # Stops the worker processes gracefully and cleans up GPU memory

        print("[ORCHESTRATOR] Sending STOP signal to workers...")
        
        # Send a "STOP" signal to the prefill queue
        self.request_queue.put(None)
        
        
        # Wait for the processes to exit
        if self.prefill_process:
            self.prefill_process.join() # Wait for prefill to exit
            print("[ORCHESTRATOR] Prefill worker joined.")
        if self.decode_process:
            self.decode_process.join() # Wait for decode to exit
            print("[ORCHESTRATOR] Decode worker joined.")
            
        self.manager.shutdown()
        print("[ORCHESTRATOR] Workers stopped and cleaned up.")

    def _send_request(self, request_id, prompt):
        #Internal method to put a request on the queue
        print(f"[ORCHESTRATOR] Sending request {request_id}...")
        self.request_queue.put((request_id, prompt))
        return request_id

    def _get_response(self, request_id, timeout=120):
        # Internal method to poll for a response
        
        start_time = time.time()
        while True:
            if request_id in self.response_dict:
                return self.response_dict.pop(request_id)
            if time.time() - start_time > timeout:
                return f"Error: Request {request_id} timed out."
            time.sleep(0.1)

    def process_request_sync(self, request_id, prompt):
        # Sends one request and waits for its specific response
        self._send_request(request_id, prompt)
        response = self._get_response(request_id)
        return response


In [26]:
# MAIN EXECUTION 
if __name__ == "__main__":
    
    try:
        token = UserSecretsClient().get_secret("HF_TOKEN")
        print("[ORCHESTRATOR] HF_TOKEN found.")
    except Exception as e:
        print(f"[ORCHESTRATOR] HF_TOKEN not found in Kaggle Secrets. Exiting. Error: {e}")
        sys.exit(1)

    # Initialize and start the API
    api = None
    try:
        api = OrchestratorAPI(hf_token=token)
        api.start_workers()
        
        total_start_time = time.time()
        
        # Simulate parallel requests with a ThreadPool
        print(f"[ORCHESTRATOR] Submitting {len(PROMPTS_TO_RUN)} requests in parallel...")
        
        request_ids = {} 
        
        with ThreadPoolExecutor(max_workers=len(PROMPTS_TO_RUN)) as executor:
            for req_id, prompt in PROMPTS_TO_RUN:
                if "long" in req_id:
                    time.sleep(0.1)
                
                # Submit the _send_request task to the thread pool
                future = executor.submit(api._send_request, req_id, prompt)
                request_ids[future] = req_id

        print("[ORCHESTRATOR] All requests submitted.")
        
        # Wait for all responses to come back
        print("[ORCHESTRATOR] Waiting for responses...")
        results = {}
        for req_id, prompt in PROMPTS_TO_RUN:
            # Poll for the result of this specific request
            result = api._get_response(req_id)
            results[req_id] = result
            print("---")
            print(f"RESULT FOR {req_id}:")
            print(f"PROMPT: {prompt[:100]}...")
            print(f"RESPONSE: {result}")
            print("---")

        total_end_time = time.time()
        
        print("\n--- FINAL REPORT (DISAGGREGATED) ---")
        print(f"Total time to process all {len(PROMPTS_TO_RUN)} requests (parallel): {total_end_time - total_start_time:.2f}s")

    except Exception as e:
        print(f"[ORCHESTRATOR] An error occurred: {e}")
    finally:
        # Clean up the worker processes
        if api:
            api.stop_workers()



[ORCHESTRATOR] HF_TOKEN found.
[ORCHESTRATOR] Initializing...
[ORCHESTRATOR] Initialization complete.
[ORCHESTRATOR] Starting worker processes...


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

[ORCHESTRATOR] Workers started. Waiting for model loading...


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

[PREFILL-WORKER-0] Initializing on cuda:0...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[DECODE-WORKER-1] Initializing on cuda:1...


Loading checkpoint shards: 100%|██████████| 4/4 [00:24<00:00,  6.02s/it]


[ORCHESTRATOR] Models should be loaded. System is READY.
[ORCHESTRATOR] Submitting 10 requests in parallel...
[ORCHESTRATOR] Sending request p1_light_light...
[ORCHESTRATOR] Sending request p2_light_light...
[ORCHESTRATOR] Sending request p3_light_heavy...
[ORCHESTRATOR] Sending request p4_light_heavy...
[ORCHESTRATOR] Sending request p5_light_heavy...
[ORCHESTRATOR] Sending request p6_heavy_light...
[ORCHESTRATOR] Sending request p7_heavy_light...
[ORCHESTRATOR] Sending request p8_heavy_light...
[ORCHESTRATOR] Sending request p9_heavy_heavy...
[ORCHESTRATOR] Sending request p10_heavy_heavy...
[ORCHESTRATOR] All requests submitted.
[ORCHESTRATOR] Waiting for responses...
---
RESULT FOR p1_light_light:
PROMPT: 
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Name three planets in our solar system th...
RESPONSE: Here are three planets in our solar system that have rings:

1. Saturn
2. Jupiter
3. Uranus
---
---
RESULT FOR p2_light_light:
PROMPT: 
<|begin_of_text|><|start_header

In [27]:
#Custom Streamer for Benchmarking
class BenchmarkStreamer:
    """
    A simple, non-printing streamer that records:
      - start_time (should be set externally right before generation starts)
      - first_token_time (set when first token arrives via put)
      - end_time (updated on every token put; final end_time is used)
      - token_count
    """
    def __init__(self):
        self.start_time = None
        self.first_token_time = None
        self.end_time = None
        self.token_count = 0

    def put(self, value):
        now = time.time()
        # Set first_token_time the moment we get the first token
        if self.first_token_time is None:
            self.first_token_time = now
            
        # update end_time and token count for each token
        self.end_time = now
        self.token_count += 1

    # optional method used by some transformers streamers (no-op)
    def end(self):
        self.end_time = time.time()

    def get_stats(self):
        """
        Returns (ttft_seconds, tpot_seconds_per_token, token_count)
        - ttft = time from start_time to first token
        - tpot = average time per-token AFTER the first token
        If start_time was not set or no tokens were produced, return zeros.
        """
        if self.start_time is None or self.first_token_time is None:
            # Not started or no tokens produced
            return 0.0, 0.0, 0

        ttft = self.first_token_time - self.start_time

        # Ensure end_time is set (if not set, assume last token time equals first token time)
        if self.end_time is None:
            self.end_time = self.first_token_time

        # TPOT = avg time per token after the first token
        if self.token_count > 1:
            decode_time_after_first = self.end_time - self.first_token_time
            tpot = decode_time_after_first / (self.token_count - 1)
        else:
            tpot = 0.0

        return ttft, tpot, self.token_count


In [31]:
# MAIN MONOLITHIC EXECUTION
if __name__ == "__main__":

    try:
        token = UserSecretsClient().get_secret("HF_TOKEN")
        os.environ["HF_TOKEN"] = token
        print("[MONOLITHIC] HF_TOKEN found.")
    except Exception as e:
        print(f"[MONOLITHIC] HF_TOKEN not found in Kaggle Secrets. Exiting. Error: {e}")
        sys.exit(1)

    # Load Model and Tokenizer on ONE GPU
    device = torch.device("cuda:1")
    print(f"[MONOLITHIC] Initializing on {device}...")
    model = None
    tokenizer = None
    try:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_quant_type="nf4",
        )
        
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID, 
            quantization_config=quantization_config,
            torch_dtype=torch.bfloat16,
            device_map=device
        )
        model.eval()
        
        tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
        
        print(f"[MONOLITHIC] Llama 3 8B (4-bit) loaded successfully on {device}.")
    except Exception as e:
        print(f"[MONOLITHIC] Failed to load model: {e}")
        sys.exit(1)

    # 3. Run workload SEQUENTIALLY
    print(f"[MONOLITHIC] Running {len(PROMPTS_TO_RUN)} requests sequentially...")
    
    total_start_time = time.time()
    
    for req_id, prompt in PROMPTS_TO_RUN:
        print(f"\n--- Processing {req_id} ---")

        # Create a new streamer for this request
        streamer = BenchmarkStreamer()
        
        streamer.start_time = time.time()
        prompt_start_time = streamer.start_time
        
        inputs = tokenizer(
            prompt,
            return_tensors="pt"
        ).to(device)
        
        # Run generation with the streamer
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKS,
                eos_token_id=tokenizer.eos_token_id,
                streamer=streamer
            )
        
        # Decode the output
        generated_ids = outputs[0][inputs.input_ids.shape[1]:]
        generated_text = tokenizer.decode(
            generated_ids,
            skip_special_tokens=True
        )
        
        prompt_end_time = time.time()
        
        #Get stats from the streamer
        ttft, tpot, token_count = streamer.get_stats()
        
        # Handle division by 0
        if tpot > 0:
            decode_tps = 1.0 / tpot
        else:
            decode_tps = 0.0

        print(f"PROMPT: {prompt[:100]}...")
        print(f"RESPONSE: {generated_text}")
        print(f"Time: {prompt_end_time - prompt_start_time:.2f}s ... TTFT: {ttft*1000:.2f} ms ({decode_tps:.1f} tok/s) | Tokens: {token_count}")
        
    total_end_time = time.time()

    print("\n--- FINAL REPORT (MONOLITHIC) ---")
    print(f"Total time to process all {len(PROMPTS_TO_RUN)} requests (sequential): {total_end_time - total_start_time:.2f}s")
    
    # Clean up GPU memory
    del model
    del tokenizer
    torch.cuda.empty_cache()
    print("[MONOLITHIC] Model unloaded and GPU memory cleaned.")



[MONOLITHIC] HF_TOKEN found.
[MONOLITHIC] Initializing on cuda:1...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[MONOLITHIC] Llama 3 8B (4-bit) loaded successfully on cuda:1.
[MONOLITHIC] Running 10 requests sequentially...

--- Processing p1_light_light ---


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

PROMPT: 
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
[CONTEXT]
In this extensive passage, we d...
RESPONSE: One commonly used activation function is the Sigmoid function.
Time: 1.43s ... TTFT: 2.53 ms (8.4 tok/s) | Tokens: 13

--- Processing p8_heavy_light ---


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


PROMPT: 
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
[CONTEXT]
You are given a long transcript...
RESPONSE: The main cause of the French Revolution was the widening economic and social disparities between the wealthy nobility and the impoverished commoners, exacerbated by the financial burdens of the American Revolutionary War and the Enlightenment ideas of liberté, égalité, fraternité, which fueled the demand for greater social and political equality.
Time: 5.64s ... TTFT: 2.76 ms (11.2 tok/s) | Tokens: 64

--- Processing p9_heavy_heavy ---


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


PROMPT: 
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
[CONTEXT]
Imagine a detailed 1,000-token ...
RESPONSE: **Policy Proposal: Accelerating the Transition to Renewable Energy in Developing Countries**

**Executive Summary**

The global imperative to mitigate climate change demands a swift and collective response. Developing countries, which are disproportionately vulnerable to the impacts of climate change, must prioritize the transition to renewable energy to ensure sustainable development and economic growth. This proposal outlines a comprehensive strategy to support the transition to renewable energy in developing countries, leveraging international cooperation, technological innovation, and socioeconomic benefits.

**I. Introduction**

The Intergovernmental Panel on Climate Change (IPCC) warns that the world has approximately 10 years to limit global warming to 1.5°C above pre-industrial levels. Developing countries, which account for approximately 80% of the world's 